<a href="https://colab.research.google.com/github/jiwon-hae/LLama-training/blob/main/(Part_6)AI_Service(Template).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# 필요 Library install
!pip install transformers==4.40.0 peft==0.10.0 trl==0.8.6 bitsandbytes==0.43.0 accelerate==0.29.0 vllm==0.4.3 gradio==4.29.0 pydantic==2.5.2

In [2]:
!pip install flash-attn==2.5.5 --no-build-isolation

In [3]:
# HF token 설정
from huggingface_hub import notebook_login
notebook_login()

In [4]:
# Google Drive Import
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel, PeftConfig
from vllm import LLM, SamplingParams
import gradio as gr

from vllm import LLM, SamplingParams

vllm_model = LLM(
    model = "Evion/llama3-alpaca-tuned-and-merged",
    tokenizer = "Evion/llama3-alpaca-tuned-and-merged",
    gpu_memory_utilization=0.85,
    enable_lora = True
)

sampling_params = SamplingParams(temperature=0.05, top_p=0.95, max_tokens=256)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:795: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/698 [00:00<?, ?B/s]

INFO 02-02 08:43:46 llm_engine.py:161] Initializing an LLM engine (v0.4.3) with config: model='Evion/llama3-alpaca-tuned-and-merged', speculative_config=None, tokenizer='Evion/llama3-alpaca-tuned-and-merged', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, rope_scaling=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.bfloat16, max_seq_len=8192, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto, quantization_param_path=None, device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='outlines'), seed=0, served_model_name=Evion/llama3-alpaca-tuned-and-merged)


tokenizer_config.json:   0%|          | 0.00/50.6k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

...

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

INFO 02-02 08:45:53 model_runner.py:146] Loading model weights took 14.9634 GB
INFO 02-02 08:45:54 gpu_executor.py:83] # GPU blocks: 8585, # CPU blocks: 2048
INFO 02-02 08:45:56 model_runner.py:854] Capturing the model for CUDA graphs. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI.
INFO 02-02 08:45:56 model_runner.py:858] CUDA graphs can take additional 1~3 GiB memory per GPU. If you are running out of memory, consider decreasing `gpu_memory_utilization` or enforcing eager mode. You can also reduce the `max_num_seqs` as needed to decrease memory usage.
INFO 02-02 08:46:05 model_runner.py:924] Graph capturing finished in 8 secs.


In [6]:
from vllm.lora.request import LoRARequest

dpo_output_1_base_path = "/content/drive/MyDrive/dpo_output_1/"
dpo_output_2_base_path = "/content/drive/MyDrive/dpo_output_2/"

lora_configs = {
    "dpo_output_1_cpk_10": (1, dpo_output_1_base_path + "checkpoint-10"),
    "dpo_output_1_cpk_20": (2, dpo_output_1_base_path + "checkpoint-20"),
    "dpo_output_1_cpk_30": (3, dpo_output_1_base_path + "checkpoint-30"),
    "dpo_output_2_cpk_10": (5, dpo_output_2_base_path + "checkpoint-10"),
    "dpo_output_2_cpk_20": (6, dpo_output_2_base_path + "checkpoint-20"),
    "dpo_output_2_cpk_30": (7, dpo_output_2_base_path + "checkpoint-30")
}

def generate_text(raw_input, temperature = 0.05, top_p = 0.95, max_tokens=256, lora_mode = "default"):
    sampling_params = SamplingParams(
        temperature=temperature,
        top_p=top_p,
        max_tokens=max_tokens
    )
    alpaca_prompt = """Below is an instruction that describes a task. Write a response that appropriately completes the request.

    ### Instruction:
    {}

    ### Response:
    {}"""
    prompt = alpaca_prompt.format(raw_input,"")
    lora_config = {}



    if lora_mode != "default":
        lora_config["lora_request"] = LoRARequest(lora_mode, lora_configs[lora_mode][0], lora_configs[lora_mode][1])

    outputs = vllm_model.generate(
        [prompt],
        sampling_params,
        **lora_config,
    )
    return outputs[0].outputs[0].text

In [8]:
!pip uninstall gradio
!pip install --upgrade gradio

Found existing installation: gradio 4.29.0
Uninstalling gradio-4.29.0:
  Would remove:
    /usr/local/bin/gradio
    /usr/local/bin/upload_theme
    /usr/local/lib/python3.11/dist-packages/gradio-4.29.0.dist-info/*
    /usr/local/lib/python3.11/dist-packages/gradio/*
  Would not remove (might be manually added):
    /usr/local/lib/python3.11/dist-packages/gradio/frpc_linux_amd64_v0.2
    /usr/local/lib/python3.11/dist-packages/gradio/launches.json
Proceed (Y/n)? Y
  Successfully uninstalled gradio-4.29.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.7/57.7 MB 38.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 321.9/321.9 kB 29.6 MB/s eta 0:00:00
  Attempting uninstall: gradio-client
    Found existing installation: gradio_client 0.16.1
    Uninstalling gradio_client-0.16.1:
      Successfully uninstalled gradio_client-0.16.1


In [9]:
iface = gr.Interface(
    fn=generate_text,
    inputs=[
        gr.Textbox(label="Prompt"),
        gr.Slider(0.0, 2.0, value=0.0, label="Temperature"),
        gr.Slider(0.0, 1.0, value=0.95, label="Top P"),
        gr.Slider(1, 1000, value=256, step=1, label="Max Tokens"),
        gr.Dropdown(
            choices=["default"] + list(lora_configs.keys()),
            label="Model Type",
            value="default"
        ),
    ],
    outputs="text",
    title="Fine-tuned PEFT Model Demo",
    description="Enter a prompt to generate text using the fine-tuned model.",
    api_name = "generate"
)

iface.launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://6286f94d6459c67a90.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


In [13]:
iface.close()

Closing server running on port: 7863
